完成外挂知识库的构建后，下一步是结合问题检索出相关上下文，并整合到提示，让llm生成回复，所以检索生成包括以下步骤：

![alt text](../../doc/image.png)

- **检索**：在向量数据库中检索出和问题相关的上下文，如Top策略，Score策略
- **后处理**：将检索出来的chunk进行后出来，包括重排名、转换或过滤
- **整合提示生成**：将最终确认的上下文和提示合并，发给llm生成回复

In [1]:
# 加载外挂知识库（向量数据库）
# 设置embedding model 
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
Settings.embed_model = OllamaEmbedding(model_name="quentinz/bge-large-zh-v1.5:latest",base_url='http://192.168.3.155:11434')

# 前面已经持久化的向量数据库
import chromadb
from llama_index.core import VectorStoreIndex
# 需要先pip install llama-index-vector-stores-chroma 
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")
# create collection
chroma_collection = db.get_or_create_collection("quickstart")
# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index=VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    storage_context=storage_context)

如果已经构建知识的索引，可以使用以下接口快速生成检索引擎，一行代码完成以上的三个步骤

In [2]:
# 设置llm
from llama_index.llms.ollama import Ollama
Settings.llm = Ollama(model="qwen2.5:latest", request_timeout=360.0,base_url='http://192.168.3.155:11434')

query_engine = index.as_query_engine()
response = query_engine.query(
    "关羽如何战胜华雄的？"
)
print(response)

关羽请战斩华雄的过程是这样的：在诸侯大会讨伐董卓时，华雄在盟军阵前挑战无人能敌。关羽主动请缨，要求出战华雄。尽管袁术看不起关羽，但曹操看中关羽的勇武，推荐他出战并递给他一杯热酒。关羽豪气十足地接过酒杯说会先斩了华雄再回来喝酒。于是上马飞奔出寨，一招便将华雄斩落马下，提着华雄的头颅回到军帐前，杯中的酒依旧烫手。


相比较使用高级接口，LlamaIndex还提供低级接口，以实现更加精细的控制

In [5]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import KeywordNodePostprocessor  

# 配置增强检索 
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# 配置上下文与prompt的合成方式 
response_synthesizer = get_response_synthesizer()

# 组合生成引擎 
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[
        KeywordNodePostprocessor(
        required_keywords=['诸葛亮','周瑜','赵云'],
        lang='zh')]
)

# 查询
response = query_engine.query("赤壁大战如何获得胜利的？")
print(response)

赤壁大战中，孙刘联军能够取得胜利的关键在于他们利用了曹操军队的战略失误和战术上的劣势。首先，诸葛亮为周瑜提供了火攻的策略，并准备了大量的火船和易燃物。其次，周瑜设下苦肉计，让黄盖诈降成功接近曹军舰队。

在具体战斗中，赵云、黄忠等将领勇猛冲锋，多次挫败曹操军队的进攻。如赵云单枪匹马立于寨外诱敌深入，最终大破曹军；黄忠与副将张著领兵烧毁了曹军粮草，削弱了敌方实力。

此外，联军还利用了天时地利。当时的天气条件非常适合实施火攻，风向对孙刘联军非常有利，而曹操的军队则处于不利位置。这些因素共同作用下，导致曹操的大批战船被焚毁，大量的士兵落水溺亡或被烧死，从而取得了赤壁之战的重大胜利。
